In [1]:
# Import of the pyomo module
from pyomo.environ import *
import numpy as np
import matplotlib.pyplot as plt

# Creation of a Concrete Model
model = ConcreteModel()

idx_end = 100
t_end = 10.0
Dt = t_end/idx_end

model.i = Set(initialize=range(idx_end), doc='tiempo')
model.j = Set(initialize=range(idx_end-1), doc='tiempo reducido')
model.k = Set(initialize=range(2), doc='tiempo reducido')

model.Df  = Var(model.i, bounds=(-1.0,1.0), doc='current')
model.p_g = Var(model.i, bounds=(-5,5), doc='voltage')
model.p_nc  = Var(model.i, bounds=(-0.0001,0.1), doc='voltage')
model.e_nc  = Var(model.i, bounds=(-10,10), doc='voltage')
model.f_min  = Var(model.i, initialize=0.01, bounds=(-10,10), doc='minimum frequency')

def _dp_g(model, i):
    T_g = 3.0
    Droop = 0.05
    
    def f(i):
        p_c = -1.0/Droop*model.Df[i]
        return 1/T_g*(p_c-model.p_g[i])
    
    return model.p_g[i+1] == model.p_g[i] + Dt*0.5*(f(i+1)+f(i))
model.veq = Constraint(model.j, rule=_dp_g, doc='generator power')

def _dDf(model, i):
    p_l = 0.1
    H = 3.5
    def f(i):
        return 1/(2*H)*(-p_l+model.p_nc[i]+model.p_g[i]-model.Df[i])    
    return model.Df[i+1] == model.Df[i] + Dt*0.5*(f(i+1)+f(i))
model.ileq = Constraint(model.j, rule=_dDf, doc='frequency increment')

def _de(model, i):
    def f(i):
        return -model.p_nc[i]    
    return model.e_nc[i+1] == model.e_nc[i] + Dt*0.5*(f(i+1)+f(i))
model.e = Constraint(model.j, rule=_de, doc='frequency increment')

def _f_min1(model, i):
    def f(i):
        #K=1
        #for k in model.Df:
            
        #if value(model.Df[i]) < -0.1:
        #    K=1
        #else:
        #    K=1
        
        u = model.Df[i]
        f_value = 1.0/0.1*(u-model.f_min[i])
        #else: 
        #    u = model.f_min[i]
        #    f_value = 1.0/0.1*(u-model.f_min[i])
        #if u == model.f_min[i]:
        #    f_value =0
        return f_value
    delta = Dt*0.5*(f(i+1)+f(i)) 
    if value(delta) >0:
        delta =0
    return model.f_min[i+1] == model.f_min[i] + delta 

def _f_min2(model, i):
    time = Dt*i
    DF = model.Df[i]
    if time>0.37:
        DF = -.005
        
    return model.f_min[i+1] == model.f_min[i] + Dt*(model.Df[i]-(DF))**2


def _f_min3(model, i):
    return model.f_min[i] == min([value(item) for item in model.Df])

def _f_min4(model, i):
    return model.f_min[i+1] == min(model.f_min[i], model.Df[i])


model.confmin = Constraint(model.j, rule=_f_min2, doc='frequency increment')

def con_desired_1(model):
    return model.e_nc[idx_end-1] == -0.04
model.desired1 = Constraint(rule=con_desired_1, doc='restr')

def con_desired_2(model):
    return model.p_nc[idx_end-1] == 0
model.desired2 = Constraint(rule=con_desired_2, doc='restr')

def initial1(model):
    return model.Df[0] == 0
model.initial1 = Constraint(rule=initial1, doc='initial value')

def initial2(model):
    return model.p_g[0] == 0
model.initial2 = Constraint(rule=initial2, doc='initial value')

def initial3(model):
    return model.e_nc[0] == 0
model.initial3 = Constraint(rule=initial3, doc='initial value')

def initial4(model):
    return model.p_nc[0] == 0
model.initial4 = Constraint(rule=initial4, doc='initial value')

def initial5(model):
    return model.f_min[0] == 0.01
model.initial5 = Constraint(rule=initial5, doc='initial value')

def objective_rule(model):
#    D = 0
#    for item in model.Df.values():
#        D+=item**2
#    print(np.min(model.Df.values()))
    #print(min(model.Df[5],model.Df[10],model.Df[50]))
    return model.f_min[idx_end-1]
model.objective = Objective(rule=objective_rule, sense=minimize, doc='Define objective function')


from pyomo.opt import SolverFactory
import pyomo.environ
opt = SolverFactory("ipopt")
results = opt.solve(model)

t = np.array([item for item in model.Df.get_values()])*Dt
Df = np.array([item for item in model.Df.get_values().values()])
p_g = np.array([item for item in model.p_g.get_values().values()])
p_nc = np.array([item for item in model.p_nc.get_values().values()])
e_nc = np.array([item for item in model.e_nc.get_values().values()])
f_min = np.array([item for item in model.f_min.get_values().values()])

In [2]:
fig, (ax0,ax1,ax2) = plt.subplots(nrows=3)   # creates a figure with one axe

ax0.plot(t,Df)
#ax0.plot(t,f_min)
ax1.plot(t,p_g, 'r', lw=1)
ax1.plot(t,p_nc, 'g', lw=1)
ax2.plot(t,e_nc, 'g', lw=1)
#ax1.plot(t,v_1)
#ax1.plot(t,v_2)
#ax1.plot(t,v_3)

ax0.grid(True)
ax1.grid(True)

plt.show()


In [3]:
t

array([ 0. ,  0.1,  0.2,  0.3,  0.4,  0.5,  0.6,  0.7,  0.8,  0.9,  1. ,
        1.1,  1.2,  1.3,  1.4,  1.5,  1.6,  1.7,  1.8,  1.9,  2. ,  2.1,
        2.2,  2.3,  2.4,  2.5,  2.6,  2.7,  2.8,  2.9,  3. ,  3.1,  3.2,
        3.3,  3.4,  3.5,  3.6,  3.7,  3.8,  3.9,  4. ,  4.1,  4.2,  4.3,
        4.4,  4.5,  4.6,  4.7,  4.8,  4.9,  5. ,  5.1,  5.2,  5.3,  5.4,
        5.5,  5.6,  5.7,  5.8,  5.9,  6. ,  6.1,  6.2,  6.3,  6.4,  6.5,
        6.6,  6.7,  6.8,  6.9,  7. ,  7.1,  7.2,  7.3,  7.4,  7.5,  7.6,
        7.7,  7.8,  7.9,  8. ,  8.1,  8.2,  8.3,  8.4,  8.5,  8.6,  8.7,
        8.8,  8.9,  9. ,  9.1,  9.2,  9.3,  9.4,  9.5,  9.6,  9.7,  9.8,
        9.9])

In [4]:
results

{'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])], 'Solver': [{'Message': 'Ipopt 3.7.1\\x3a Optimal Solution Found', 'Error rc': 0, 'Time': 0.05650973320007324, 'Id': 0, 'Status': 'ok', 'Termination condition': 'optimal'}], 'Problem': [{'Sense': 'unknown', 'Lower bound': -inf, 'Upper bound': inf, 'Number of constraints': 403, 'Number of variables': 500, 'Number of objectives': 1}]}

In [5]:
min([1,2,3])

1

In [6]:
import time,sys
from IPython.display import display, clear_output
for i in range(10):
    time.sleep(0.25)
    clear_output(wait=True)
    print(i)
    sys.stdout.flush()

9
